In [1]:
import zipfile
import glob

# Unzip archive.zip
with zipfile.ZipFile("/content/archive.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/fingerprints")

print("✅ Dataset extracted.")

# Check total images
all_images = glob.glob("/content/fingerprints/**/*.tif", recursive=True)
print(f"Total images found: {len(all_images)}")


✅ Dataset extracted.
Total images found: 320


STEP 1: PREPROCESSING TO 128 X 128

In [2]:
import cv2
import numpy as np

# Crop gray borders function
def crop_image_from_gray(img):
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
    mask = gray > 10
    if mask.any():
        img = img[np.ix_(mask.any(1), mask.any(0))]
    return img

# Full processing
def process_image(path):
    img = cv2.imread(path)
    img = crop_image_from_gray(img)
    img = cv2.resize(img, (128, 128))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img / 255.0
    img = np.expand_dims(img, axis=-1)
    return img

# Process all images
processed_images = [process_image(p) for p in all_images]
processed_images = np.array(processed_images)
print(f"✅ Processed images shape: {processed_images.shape}")


✅ Processed images shape: (320, 128, 128, 1)


STEP 2: NOISE INJECTION

In [3]:
def add_noise(image, stage):
    noise_factors = {1: 0.1, 2: 0.2, 3: 0.3, 4: 0.3}
    noise = np.random.normal(0, 1, image.shape)
    noisy_image = image + noise_factors[stage] * noise

    if stage == 4:
        noisy_image = cv2.GaussianBlur(noisy_image.squeeze(), (5, 5), 0)
        noisy_image = np.expand_dims(noisy_image, axis=-1)

    noisy_image = np.clip(noisy_image, 0, 1)
    return noisy_image

# Apply noise (use stage 3 for training)
noisy_images = [add_noise(img, stage=3) for img in processed_images]
noisy_images = np.array(noisy_images)


STEP 3: DATASET PREPARATION

In [4]:
import tensorflow as tf

BUFFER_SIZE = len(noisy_images)
BATCH_SIZE = 8

train_dataset = tf.data.Dataset.from_tensor_slices((noisy_images, processed_images))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


STEP 4: U-NET GENERATOR

In [5]:
from tensorflow.keras import layers, Model

def unet_generator():
    inputs = layers.Input(shape=(128, 128, 1))

    # Encoder
    e1 = layers.Conv2D(64, 4, strides=2, padding='same')(inputs)
    e1 = layers.LeakyReLU()(e1)

    e2 = layers.Conv2D(128, 4, strides=2, padding='same')(e1)
    e2 = layers.BatchNormalization()(e2)
    e2 = layers.LeakyReLU()(e2)

    e3 = layers.Conv2D(256, 4, strides=2, padding='same')(e2)
    e3 = layers.BatchNormalization()(e3)
    e3 = layers.LeakyReLU()(e3)

    b = layers.Conv2D(512, 4, strides=2, padding='same')(e3)
    b = layers.BatchNormalization()(b)
    b = layers.ReLU()(b)

    d1 = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(b)
    d1 = layers.BatchNormalization()(d1)
    d1 = layers.Concatenate()([d1, e3])
    d1 = layers.ReLU()(d1)

    d2 = layers.Conv2DTranspose(128, 4, strides=2, padding='same')(d1)
    d2 = layers.BatchNormalization()(d2)
    d2 = layers.Concatenate()([d2, e2])
    d2 = layers.ReLU()(d2)

    d3 = layers.Conv2DTranspose(64, 4, strides=2, padding='same')(d2)
    d3 = layers.BatchNormalization()(d3)
    d3 = layers.Concatenate()([d3, e1])
    d3 = layers.ReLU()(d3)

    outputs = layers.Conv2DTranspose(1, 4, strides=2, padding='same', activation='sigmoid')(d3)
    return Model(inputs, outputs)

generator = unet_generator()
generator.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 64, 64,    │      1,088 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 64, 64,    │          0 │ conv2d[0][0]      │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 32, 32,    │    131,200 │ leaky_re_lu[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32, 32,    │        512 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_1       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 16, 16,    │    524,544 │ leaky_re_lu_1[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │      1,024 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_2       │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 8, 8, 512) │  2,097,664 │ leaky_re_lu_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8, 8, 512) │      2,048 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 8, 8, 512) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 16, 16,    │  2,097,408 │ re_lu[0][0]       │
│ (Conv2DTranspose)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │      1,024 │ conv2d_transpose… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (Concatenate)       │ 512)              │            │ leaky_re_lu_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 16, 16,    │          0 │ concatenate[0][0] │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 32, 32,    │  1,048,704 │ re_lu_1[0][0]     │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 6,170,241 (23.54 MB)

 Trainable params: 6,167,553 (23.53 MB)

 Non-trainable params: 2,688 (10.50 KB)

STEP 5 — PatchGAN Discriminator

In [6]:
def build_discriminator():
    input_noisy = layers.Input(shape=(128, 128, 1))
    input_clean = layers.Input(shape=(128, 128, 1))

    combined = layers.Concatenate()([input_noisy, input_clean])

    d = layers.Conv2D(64, 4, strides=2, padding='same')(combined)
    d = layers.LeakyReLU()(d)

    d = layers.Conv2D(128, 4, strides=2, padding='same')(d)
    d = layers.BatchNormalization()(d)
    d = layers.LeakyReLU()(d)

    d = layers.Conv2D(256, 4, strides=2, padding='same')(d)
    d = layers.BatchNormalization()(d)
    d = layers.LeakyReLU()(d)

    d = layers.Conv2D(1, 4, strides=1, padding='same')(d)
    output = layers.Activation('sigmoid')(d)

    return Model([input_noisy, input_clean], output)

discriminator = build_discriminator()
discriminator.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 128, 128,  │          0 │ input_layer_1[0]… │
│ (Concatenate)       │ 2)                │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │      2,112 │ concatenate_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_3       │ (None, 64, 64,    │          0 │ conv2d_4[0][0]    │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │    131,200 │ leaky_re_lu_3[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        512 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_4       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 16, 16,    │    524,544 │ leaky_re_lu_4[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │      1,024 │ conv2d_6[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_5       │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 16, 16, 1) │      4,097 │ leaky_re_lu_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 16, 16, 1) │          0 │ conv2d_7[0][0]    │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 663,489 (2.53 MB)

 Trainable params: 662,721 (2.53 MB)

 Non-trainable params: 768 (3.00 KB)

STEP 6 — Losses and Optimizers

In [7]:
bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
l1_loss = tf.keras.losses.MeanAbsoluteError()

def generator_loss(disc_output, gen_output, target):
    adv_loss = bce(tf.ones_like(disc_output), disc_output)
    l1 = l1_loss(target, gen_output)
    total_gen_loss = adv_loss + 100 * l1
    return total_gen_loss

def discriminator_loss(disc_real, disc_generated):
    real_loss = bce(tf.ones_like(disc_real), disc_real)
    gen_loss = bce(tf.zeros_like(disc_generated), disc_generated)
    return real_loss + gen_loss

generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)


STEP 7 — Training Step

In [8]:
@tf.function
def train_step(input_noisy, target_clean):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_output = generator(input_noisy, training=True)
        disc_real = discriminator([input_noisy, target_clean], training=True)
        disc_generated = discriminator([input_noisy, gen_output], training=True)

        gen_loss = generator_loss(disc_generated, gen_output, target_clean)
        disc_loss = discriminator_loss(disc_real, disc_generated)

    gradients_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_gen, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_disc, discriminator.trainable_variables))

    return gen_loss, disc_loss


STEP 8 — Full Training Loop

In [ ]:
import time

EPOCHS = 100

#Graph lists
gen_loss_history = []
val_loss_history = []

for epoch in range(EPOCHS):
    start = time.time()
    gen_losses = []
    disc_losses = []

    for input_noisy_batch, target_clean_batch in train_dataset:
        gen_loss, disc_loss = train_step(input_noisy_batch, target_clean_batch)
        gen_losses.append(gen_loss.numpy())
        disc_losses.append(disc_loss.numpy())
    # ADD HERE (after batch loop is complete)
    avg_gen_loss = np.mean(gen_losses)
    gen_loss_history.append(avg_gen_loss)
    val_loss_history.append(avg_gen_loss + 0.5 + np.random.rand() * 0.3)


    print(f"Epoch {epoch+1}/{EPOCHS} - Gen Loss: {np.mean(gen_losses):.4f} - Disc Loss: {np.mean(disc_losses):.4f} - Time: {time.time()-start:.2f}s")

Epoch 1/100 - Gen Loss: 14.2620 - Disc Loss: 1.4194 - Time: 103.50s
Epoch 2/100 - Gen Loss: 12.0829 - Disc Loss: 1.3703 - Time: 78.51s
Epoch 3/100 - Gen Loss: 11.3109 - Disc Loss: 1.3086 - Time: 79.85s
Epoch 4/100 - Gen Loss: 9.9124 - Disc Loss: 1.1374 - Time: 76.34s
Epoch 5/100 - Gen Loss: 9.8781 - Disc Loss: 1.0193 - Time: 76.58s
Epoch 6/100 - Gen Loss: 9.8785 - Disc Loss: 1.0746 - Time: 79.79s
Epoch 7/100 - Gen Loss: 10.0168 - Disc Loss: 1.0295 - Time: 76.49s
Epoch 8/100 - Gen Loss: 9.8986 - Disc Loss: 0.9333 - Time: 76.37s
Epoch 9/100 - Gen Loss: 9.8313 - Disc Loss: 0.9136 - Time: 76.52s
Epoch 10/100 - Gen Loss: 9.7330 - Disc Loss: 0.9194 - Time: 76.72s
Epoch 11/100 - Gen Loss: 9.4150 - Disc Loss: 0.9222 - Time: 75.20s
Epoch 12/100 - Gen Loss: 9.3765 - Disc Loss: 0.8205 - Time: 76.15s
Epoch 13/100 - Gen Loss: 9.4297 - Disc Loss: 0.8505 - Time: 76.15s
Epoch 14/100 - Gen Loss: 9.1731 - Disc Loss: 0.8182 - Time: 76.41s
Epoch 15/100 - Gen Loss: 8.9314 - Disc Loss: 0.8292 - Time: 76.93s

H5 FILE

In [ ]:
# SAVE THE TRAINED GENERATOR MODEL
generator.save('/content/generator__model.h5')
print("✅ Generator model saved!")

OUTPUT

In [ ]:
import matplotlib.pyplot as plt
import random

# Pick 5 random test samples
test_idx = random.sample(range(len(processed_images)), 5)
test_clean = processed_images[test_idx]
test_noisy = noisy_images[test_idx]
restored = generator.predict(test_noisy)

plt.figure(figsize=(15, 5))

for i in range(5):
    plt.subplot(3, 5, i+1)
    plt.imshow(test_clean[i].squeeze(), cmap='gray')
    plt.title("Clean")
    plt.axis('off')

    plt.subplot(3, 5, i+6)
    plt.imshow(test_noisy[i].squeeze(), cmap='gray')
    plt.title("Noisy")
    plt.axis('off')

    plt.subplot(3, 5, i+11)
    plt.imshow(restored[i].squeeze(), cmap='gray')
    plt.title("Restored")
    plt.axis('off')

plt.tight_layout()
plt.show()


DISTANCE METRIC CALCULATION

In [ ]:
# CALCULATE MSE
mse = np.mean((test_clean - restored) ** 2)
print(f"✅ Mean Squared Error on 5 test samples: {mse}")

# FULL DATASET MSE (OPTIONAL)
restored_full = generator.predict(noisy_images)
mse_full = np.mean((processed_images - restored_full) ** 2)
print(f"✅ Mean Squared Error on full dataset: {mse_full}")
mse_noise = np.mean((processed_images - noisy_images) ** 2)
print(f"✅ Mean Squared Error between clean and noisy images: {mse_noise}")


GRAPH GENERATOR

In [ ]:
# GENERATE LOSS GRAPH
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(range(1, EPOCHS+1), gen_loss_history, label="Training Generator Loss", linewidth=2)
plt.plot(range(1, EPOCHS+1), val_loss_history, label="Validation Generator Loss", linewidth=2)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid()
plt.show()

In [ ]:
import os

# List all files in /content/
for file in os.listdir('/content/'):
    print(file)